# Detection Efficiency Filter Demo

This notebook demonstrates the detection efficiency/fading function filter.

In [ ]:
from sorcha.modules import PPFootprintFilter as fp
from sorcha.modules import PPFadingFunctionFilter as ff
from sorcha.modules.PPModuleRNG import PerModuleRNG

In [ ]:
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
def getSqlData(database,rows_start,nrows):
    """Import dataset from local SQL database
    
    Parameters:
    -----------
    database   ... path to database
    rows_start ... number of row to start
    rows_max   ... number of rows to select
    
    Returns:
    --------
    observations ... pandas dataframe containing observations from JPL database
    """
    con = sql.connect(database)
    observations = pd.read_sql("""SELECT *, observationStartMJD as observationStartMJD_TAI FROM observations LIMIT """+str(rows_start)+','+str(nrows), con)
    return observations

In [ ]:
def randobs(ronsky=3,ra=180,dec=45, n=100000):

    """Create random observations centered on RA, Dec with radius r.
    Random observations are generated correctly only for declinations < dec+r.
    
    Parameters:
    ------------
    ronksy ... on sky radius [deg]
    ra     ... Right Ascension of center [deg]
    dec    ... Declination of center
    
    Returns:
    --------
    raout ... Right Ascension of fake observations
    decout ... Declination of fake observations
    
    """
    # radius of the circle
    # center of the circle (x, y)
    # random angle
    
    rnd1=np.random.rand(n)
    rnd2=np.random.rand(n)
    
    alpha = 2 * np.pi * rnd1
    # random radius
    r = ronsky * np.sqrt(rnd2)
    # calculating coordinates
    raout = r * np.cos(alpha) + ra
    decout = r * np.sin(alpha) + dec
    
    return np.mod(raout+360,360),decout

## Import LSST Opsim database 
This database contains LSST pointings and environmental information such as seeing.


In [ ]:
db_path="oneline_v2.0.db"

Select the first exposure of the LSST survey and initialise number of simulated detections.

In [ ]:
LSSTdf=getSqlData(db_path,0,1)
rng=PerModuleRNG(2021)
nobs_per_field=1000000

In [ ]:
LSSTdf

Some random measurements are then created.

In [ ]:
magbins=[]
i=22.0
while(i<=26.0):
    magbins.append(i)
    i=i+0.05

rao,deco=randobs(ronsky=2.5,ra=LSSTdf['fieldRA'][0],dec=LSSTdf['fieldDec'][0],n=nobs_per_field)
dfobs0=pd.DataFrame(np.array([rao,deco]).T,columns=['AstRA(deg)','AstDec(deg)'])
dfobs0['FieldMJD_TAI']=LSSTdf['observationStartMJD_TAI'][0]
dfobs0['FieldID']=LSSTdf['observationId'][0]

In [ ]:
dfobs0

## Create magnitudes

In [ ]:
mags=np.arange(nobs_per_field)
mags=np.random.uniform(22.0,26.0,len(dfobs0))
dfobs = pd.merge(dfobs0, LSSTdf, left_on="FieldID", right_on="observationId", how="left")

# Match keywords
dfobs['observedPSFMag'] = mags
dfobs['fiveSigmaDepthAtSource']=dfobs['fiveSigmaDepth']


In [ ]:
mags

In [ ]:
dfobs['observedPSFMag']

In [ ]:
dfobs

In [ ]:
magcounts,magpins=np.histogram(dfobs['observedPSFMag'],bins=magbins)

reduced_dfobs=ff.PPFadingFunctionFilter(dfobs, 1.0, 0.1, rng)

redmagcounts,redmagbins=np.histogram(reduced_dfobs['observedPSFMag'],bins=magbins)

res=redmagcounts/magcounts

fiveSigma=dfobs.fiveSigmaDepthAtSource.values[0]

restheor=[]
i=0
while(i<len(magbins)):
    a=1./(1.+np.exp((magbins[i]-fiveSigma)/.1))
    restheor.append(a)
    i=i+1

plt.clf()
plt.scatter(magpins[:-1]+0.025,res)
plt.plot(magbins,restheor, 'r-')
plt.xlabel('Magnitude')
plt.ylabel('Observed fraction')
#plt.savefig('deteff_.jpg', bbox_inches='tight')
plt.show()

